# Cеточные модели уравнений с частными производными

## Лабораторная №3 (Метод Адамса)

Выполнил Мусатов Данила Юрьевич КМБО-03-18

# Задание №3

### Метод Адамса
Метод Адамса относится к многошаговым методам. То есть для того, чтобы вычислить значение функции на данном слое, необходимо знать значения функции на нескольких предыдущих слоях. Представим решение дифференциального уравнения в виде
$$
y_{k+1}=y_{k}+\int_{x_{k}}^{x_{k=1}} f(x, y(x)) d x
$$

Если решение задачи Коши получено в уз.лах вплоть до $k$-го, то можно аппроксимировать подынтегральную функцию, например, интерпояционным многочленом какой-либо степени. Вычислив интеграл от построенного многочлена на отрезке $\left[x_{k}, x_{k+1}\right]$, получим ту или иную формулу А дамса. При использовании интерполяционного многочлена третьей степени, построенного по значениям подынтегральной функции в последних четырех узлах, получим метод Адамса четвертого порядка точности:
$$
y_{k+1}=y_{k}+\frac{h}{24}\left(55 f_{k}-59 f_{k-1}+37 f_{k-2}-9 f_{k-3}\right)
$$
где $f_{k}-$ значние подынтегральной функции в узле $x_{k} .$ Для использования данного метода необходимо знать значения функции на четырех предыдущих слоях, их можно определить, воспользовавшись какимнибудь подходящим одношаговым методом, например - методом РунгеКутты четвертого порядка.

#### Задание:
Решить задачу Коши методом Адамса со значениями шага h1 = 0.1, h2 = 0.01. Сравнить результаты графически
 

#### Вариант 2: $\quad y^{\prime}=\dfrac{y(2y\ln(x)-2)}{x}, \quad y(1)=\dfrac{1}{3}, \quad x \in[1,2]$ .

### Аналитическое решение

$$y{\left(1 \right)} = \frac{1}{3}$$<br><br>$$y{\left(x \right)} = \frac{2}{C_{1} x^{2} + 2 \ln{\left(x \right)} + 1}$$<br><br>$$\frac{1}{3} = \frac{2}{1^{2} C_{1} + 2 \ln{\left(1 \right)} + 1}$$<br><br>$$C_{1} = 5$$<br><br>$$y{\left(x \right)} = \frac{2}{5 x^{2} + 2 \ln{\left(x \right)} + 1}$$<br><br><a data-pod="funcgraphs.xy" data-input='{"function": "2/(1 + 2*log(x) + 5*x^2)", "x": "x"}' style="display: none;" >

## Численное решение

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as ip_w 


A, B = 1, 2
func = lambda x, y: y*(2*y*np.log(x)-2)/x # определение данной функции (производная из варианта 3)
source_func = lambda x: 2/(5*(x**2)+2*np.log(x)+1) # точное решение
Y0 = 1/3

In [2]:
def Method_Adams(func, y0, a, b, h):
    xs = np.arange(a, b+h, h) # определение точек x
    # определение коэффициентов K
    K1 = lambda x, y: func(x, y)
    K2 = lambda x, y, h: func(x + h/2, y + h*K1(x, y)/2)
    K3 = lambda x, y, h: func(x + h/2, y + h*K2(x, y, h)/2)
    K4 = lambda x, y, h: func(x + h, y + h*K3(x, y, h))

    yk_plus_1 = y0 # начальное значение
    ys = [yk_plus_1] # список с значениями y


    # рассчет согласно формулам (первые 3 рассчитываются методом Рунге-Кутты 4 порядка)
    for index, i in enumerate(xs[1:]):
        if index < 4:
            yk_plus_1 = yk_plus_1 + (h/6) * (K1(i, yk_plus_1) + 2*K2(i, yk_plus_1, h) + 2*K3(i, yk_plus_1, h) + K4(i, yk_plus_1, h))
        else:
            yk_plus_1 = yk_plus_1 + (h/24) * (55 * func(i, ys[-1]) - 59*func(i, ys[-2]) + 37*func(i, ys[-3]) -9*func(i, ys[-4]))
        ys.append(yk_plus_1)
    return xs, np.array(ys)

In [3]:
# Функция для оценки максимального отклонения от аналитического решения
def max_error(xs, ys, source_func):
    return max(np.fabs(ys - source_func(xs)))

In [4]:
def comparison_sorce_Method_Adams(y0, a, b, h):
    xs, ys = Method_Adams(func, y0, a, b, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Метод Адамса " + str(h))
    plt.plot(xs, source_func(xs), color="red", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys, source_func))

In [5]:
ip_w.interact(comparison_sorce_Method_Adams,  y0 = ip_w.fixed(Y0), a= ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

## Оценка погрешности с помощью правила Рунге

$$error = \frac{| y_h - y_{h/2}|}{2^p - 1}$$ 

In [6]:
def Runge_rule(ys1, ys2, p=1):
    return max(np.fabs(ys1 - ys2))/(2**p - 1)

Метод Адамса имеет 4-ый порядок точности

In [7]:
def Method_Adams_estimation_error_Ruge_depending_on_h(y0,a,b,h):
    _, ys1 = Method_Adams(func, y0, a, b, h)
    _, ys2 = Method_Adams(func, Y0, A, B, h/2)
    if(len(ys1)!=len(ys2[::2])):
        ys1=np.delete(ys1,len(ys1)-1,0)
    err1 = Runge_rule(ys1, ys2[::2],4)
    print(f" Ошибка по Правилу рунге для шага {h} равна {round(err1, 7)}" )

In [8]:
ip_w.interact(Method_Adams_estimation_error_Ruge_depending_on_h,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), p=ip_w.fixed(1), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Ошибка по Правилу рунге для шага 0.01 равна 0.000025
Ошибка по Правилу рунге для шага 0.1 равна 0.0004543

## Нахождение решения с помощью библиотеки scipy

In [9]:
from scipy.integrate import odeint as ODE

In [10]:
g = lambda y, x : func(x,y)
def scipy_ODE(g, y0, a, b, h):
    xs = np.arange(a, b+h, h)
    ys = ODE(g, y0, xs)
    return xs, ys

In [11]:
def comparison_source_with_Scipy(y0, a, b, h):
    xs, ys = scipy_ODE(g, Y0, A, B, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Решение с помощью модуля scipy " + str(h))
    plt.plot(xs, source_func(xs), color="red", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()    
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys.flat, source_func))

In [12]:
ip_w.interact(comparison_source_with_Scipy,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Значения от аналитического решения отличаются на 10^(-8)